# Reading the rest of HYDRUS files [Stage 2]
![The project files](../Assets/Project_files_2.png)

## Importing and loading dataset

In [1]:
import os
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from NesrHydrusAnalyst import *

In [2]:
src = '../Datasets/H3D2_SandDitch0011'

## Unreadable files
Encrypted files that are converted to text elswhere
1. `h.out`
2. `Q.out`
3. `v.out`
3. `th.out`
4. `MESHTRIA.000`
4. `DOMAIN.IN`

## Reading previous implemented files
### Reading the _mesh files_
1. `H.TXT`
2. `V.TXT`
3. `TH.TXT`
4. `MESHTRIA.TXT`
### Spread-info files

1. `Cum_Q.out`
1. `h_Mean.out`
1. `v_Mean.out`
5. `Run_Inf.out`
### the _information files_
1. `A_Level.out`
2. `ATMOSPH.IN`
3. `Balance.out`
4. `DIMENSIO.IN`
5. `Run_Inf.out` (Repeated)
6. `SELECTOR.IN`

## Files to be implemented here


### Flux-info files

1. `Boundary.out`
1. `BOUNDARY.IN`
1. `Check.out`

In [3]:
# def read_hydrus_data(folder='Current', save_to_csv=True,
#                      read_velocities=False):
#     '''
#     A function to read both Theat and H files from HYDRUS outputs, 
#         then to:
#             1- return one dataframe contains both data in a decent format.
#             2- save this output to a CSV file (optional, True by default)
#     Input:
#         The name of the main folder (leave balank for the current folder)
#         The option to save_to csv, default =True (Boolean)
#     '''
#     # Specify the source folder
#     if folder == 'Current':
#         read_dir = os.getcwd()
#     else:
#         read_dir = folder

#     # Finding number of nodes in the file
#     mesh_file = os.path.join(read_dir, 'MESHTRIA.TXT')
#     num_cells = np.array(linecache.getline(mesh_file, 6).split(), int)[0]
#     # Define dataframe titles
#     titles = ['n', 'x', 'y', 'z']
#     # Define a list of coordinates
#     full_data = [[0, 0, 0, 0]]
#     # Set a loop to geather all coordinates from MESHTRIA.TXT file
#     for i in range(8, num_cells + 8):
#         full_data.append(
#             np.array(linecache.getline(mesh_file, i).split(), float))
#     # Convert the list to numpy array then to a dataframe
#     coordinates_df = pd.DataFrame(np.array(full_data), columns=titles)

#     # Print head and tail of the dataframe to ensure correctness
#     # pd.concat([coordinates_df.head(),coordinates_df.tail()])

#     # -----------------------------#
#     # To get data from all files   #
#     # -----------------------------#
#     def get_data_from_file(filename='TH.TXT', main_caption='Theta'):
#         '''
#         Function to combine all values of a property to a single dataframe 
#         inputs:
#         filename, the name of the file
#         caption, the leading caption of the columns (we will add the portion '_T= xxx')
#         where xxx is the timestep
#         '''
#         # compute number of lines for each timestep
#         num_lines = int(math.ceil(num_cells / 10.))
#         time_steps_remaining = True  # Flag to see if the loop should continue or not.
#         times_df = pd.DataFrame([])  # Empty dataframe
#         time_loc_start = 2  # The starting cell of the timestep
#         # Check if it is a velocity file
#         processing_velocity = (filename[-5:] == 'V.TXT')
#         while time_steps_remaining:
#             line_t = linecache.getline(filename, time_loc_start).split()
#             # Check if it is the start of the timestep, otherwise exit
#             if line_t[0] == 'Time':
#                 t = int(line_t[2])
#                 if processing_velocity:
#                     velocity_component = {
#                         'first': '1',
#                         'second': '2',
#                         'third': '3'
#                     }[line_t[5].strip()]
#                     caption = main_caption + velocity_component
#                 else:
#                     caption = main_caption
#                 # Finding the last line of the timestep
#                 tim_loc_end = num_lines + time_loc_start + 2
#                 # The starting time is always 0 because steps starts in 1 in HYDRUS
#                 time_data = [0]
#                 # Create the timestep as one long list
#                 for i in range(time_loc_start + 2, tim_loc_end):
#                     time_data.extend(linecache.getline(filename, i).split())
#                 # Convert the list to DataFrame
#                 dft = pd.DataFrame(
#                     np.array(time_data, float),
#                     columns=['{}_T{}'.format(caption, t)])
#                 if len(times_df) == 0:  # If it is the first timestep
#                     times_df = dft.copy()
#                 else:  # Otherwise (for all other timesteps)
#                     times_df = pd.concat([times_df, dft], axis=1)
#                 # Change the start to the probable next timestem (if exist)
#                 time_loc_start = tim_loc_end + 1
#                 time_steps_remaining = True if len(
#                     linecache.getline(filename, time_loc_start)) > 0 else False
#                 # End IF
#         return times_df

#     # Set the basic dataframe to the coordinates dataframe, to append to it.
#     full_df = coordinates_df
#     # Looping through the basic output files then to concatenate them all
#     properties = [('TH.TXT', 'Th'), ('H.TXT', 'H'), ('V.TXT', 'V')]
#     if not read_velocities:
#         properties = properties[:-1]
#     for prop in properties:
#         file_path = os.path.join(read_dir, prop[0])
#         # Check if the file exists
#         if os.path.isfile(file_path):
#             prop_df = get_data_from_file(file_path, prop[1])
#             full_df = pd.concat([full_df, prop_df], axis=1)
#         else:
#             print(
#                 'Warning, the file {} does not exist in the given path'.format(
#                     prop[0]))

#     # Convert the num column to integer
#     full_df[['n']] = full_df[['n']].astype(np.int64)
#     # dropping the first row (the zeros row) as it is not necessary
#     full_df.drop(0, inplace=True)
#     # Saving the resultant dataframe to disk.
#     if save_to_csv:
#         full_df.to_csv(os.path.join(read_dir, 'nesr_data2.csv'))
#     return full_df

## Reading the `BOUNDARY.IN` file

In [4]:
folder =src

# def read_boundary_data(folder='Current', save_to_csv=True,
#                      read_velocities=False):
#     '''
#     A function to read both Theat and H files from HYDRUS outputs, 
#         then to:
#             1- return one dataframe contains both data in a decent format.
#             2- save this output to a CSV file (optional, True by default)
#     Input:
#         The name of the main folder (leave balank for the current folder)
#         The option to save_to csv, default =True (Boolean)
#     '''

# Specify the source folder
if folder == 'Current':
    read_dir = os.getcwd()
else:
    read_dir = folder



# Finding number of nodes in the file
mesh_file = os.path.join(read_dir, 'BOUNDARY.IN')
# num_cells = np.array(linecache.getline(mesh_file, 6).split(), int)#[0]
linecache.getline(mesh_file, 4).split()
# # Define dataframe titles
# titles = ['n', 'x', 'y', 'z']
# # Define a list of coordinates
# full_data = [[0, 0, 0, 0]]
# # Set a loop to geather all coordinates from MESHTRIA.TXT file
# for i in range(8, num_cells + 8):
#     full_data.append(
#         np.array(linecache.getline(mesh_file, i).split(), float))
# # Convert the list to numpy array then to a dataframe
# coordinates_df = pd.DataFrame(np.array(full_data), columns=titles)

['518', '0', 'f', 't', 'f', 'f']

In [5]:
folder = src

# def read_boundary_data(folder='Current', save_to_csv=True,
#                      read_velocities=False):
#     '''
#     A function to read both Theat and H files from HYDRUS outputs,
#         then to:
#             1- return one dataframe contains both data in a decent format.
#             2- save this output to a CSV file (optional, True by default)
#     Input:
#         The name of the main folder (leave balank for the current folder)
#         The option to save_to csv, default =True (Boolean)
#     '''

# Specify the source folder
if folder == 'Current':
    read_dir = os.getcwd()
else:
    read_dir = folder

# Finding number of nodes in the file
mesh_file = os.path.join(read_dir, 'BOUNDARY.IN')
num_cells = int(linecache.getline(mesh_file, 4).split()[0])
# Define dataframe titles
titles = ['n', 'surface_area']


def get_num_lines(num_cells, nums_per_line=10):
    num_lines = int(num_cells / nums_per_line)
    if num_cells % nums_per_line > 0: num_lines += 1
    return num_lines


def read_snakey_list(file_name, start_line, end_line, data_type=float):
    '''
    Reading a list of numbers thar are stored in a text file, where
    there are `numbers_count` numbers, stored as `nums_per_line` numbers 
    per line, the starting position of first occurence of numbers is at
    `start_line` line number
    returns a numpy array of the data with the `data_type` given
    '''
    points = []
    for i in range(start_line, end_line):
        points.extend(linecache.getline(file_name, i).split())
    return np.array(points, data_type)


num_lines = get_num_lines(num_cells, 10)

first_line = 10
end_line = first_line + num_lines
points = read_snakey_list(mesh_file, first_line, end_line, data_type=int)

first_line = end_line + 1
end_line = first_line + num_lines
areas = read_snakey_list(mesh_file, first_line, end_line)
data = np.array([points, areas]).T
data_in = pd.DataFrame(
    data, columns=titles).astype({
        "n": int,
        "surface_area": float
    })
data_in

,n,surface_area
0,1,0.020491
1,8,0.040982
2,9,0.762819
3,11,1.275030
4,12,0.512820
...,...,...
513,3152,1.538460
514,3153,1.538460
515,3154,1.538460
516,3155,1.538460


## Change the `get_means_table` function to read timed files

In [6]:
def get_means_table(filename,
                    header_location,
                    data_location,
                    units_location=None,
                    reading_end='end', 
                    replace_units=True):
    '''
    Returns a table of adjusted data to numeric analysis
    
    We have to specify the:
    header_location: the line number that contains the headers, 
    units_location : the line number that contains the units (if None, then
                        no units is allowed), 
    data_location  : the line number that contains the first line of data.
    
    if reading_end='end', it reads the lines until it finds the word 'end'
        (or any other word) at the beginining of the last line (it will 
        read before the 'end')
    if reading_end=integer, then it will read until it reaches the specified 
        number (inclusive)
        
    if replace_units=True, it will replace the units by equivalent numbers
        according to the following:
        'f', 't' >> 0, 1
        'mm', 'cm', 'm' >> 0, 1, 2
        'sec', 'min', 'hours', 'days', 'years' >> 0, 1, 2, 3, 4
        's', 'min', 'h', 'd', 'y' >>  0, 1, 2, 3, 4
    
    Units will be written after an underscore without any special chars
    e.g. velocity: m/s  --> velocity_mps
    
    # filename = os.path.join(file_path, 'Cum_Q.out')
    '''

    # Defining some functions
    def proper_type(x):
        try:
            nf = float(x)
            ni = float(int(nf))
            # print(nf, ni, abs(nf - ni))
            if abs(nf - ni) < 0.0000000000001:
                return int(ni)
            else:
                return nf
        except:
            return x

    def replace_text(x):
        if x in ('t', 'f'):
            # return {'t':1, 'f':0}[x]
            return ['f', 't'].index(x)
        elif x in ('mm', 'cm', 'm'):
            return ['mm', 'cm', 'm'].index(x)
        elif x in ('sec', 'min', 'hours', 'days', 'years'):
            return ['sec', 'min', 'hours', 'days', 'years'].index(x)
        elif x in ('s', 'min', 'h', 'd', 'y'):
            return ['s', 'min', 'h', 'd', 'y'].index(x)
        else:
            return x  # proper_type(x)

    def get_line(filename, pos, replace_units=True):
        line_feed = linecache.getline(filename, pos).split()
#         print(line_feed)
        if replace_units:
            return list(map(replace_text, line_feed))
        else:
            return line_feed

    def get_word(filename, pos, loc=0):
        word = get_line(pos)
        if len(word) < 1:
            return ''
        else:
            word = word[loc]
        if isinstance(word, str):
            return word.strip()
        else:
            return word

    def get_num(p1, p2, is2d):
        '''
        p1, the line of 2D file
        p2, the line of 3D file
        '''
        return {True: p1, False: p2}[is2d]

    def adjust_body(replaceable, headers, body):
        for _ in range(len(headers) - len(body)):
            body.append(replaceable)

    def reform_unit(unit, prefix=None):
        '''
        Functions to add units

        '''

        def split_letters(word):
            return [_ for _ in word]

        reformed = ''.join([{
            '/': 'p',
            '.': '',
            '[': '',
            ']': ''
        }.get(i, i) for i in split_letters(unit)])
        if prefix is None:
            return reformed
        else:
            return prefix + reformed

    # Defining variables
    headers = []
    body = []
    headers += get_line(filename, header_location, False)
#     print(headers)
    try:
        # If there is any empty columns, remove them
        for _ in range(5):
            headers.remove('...')
    except:
        pass
#     print(headers)
    headers = [_.replace('...', '') for _ in headers]

    if units_location is not None:
        units = get_line(filename, units_location, replace_units)
        units = [reform_unit(_, '_') for _ in units]
        headers = [x + y for x, y in zip(headers, units)]
    
    if type(reading_end)==str: # 'end' for example
        # reading to the end of the file
        i = data_location
        feed = get_line(filename, i, replace_units)
        while feed[0] != reading_end:
            body.append(feed)
            i += 1
            feed = get_line(filename, i, replace_units)
    else:
        for i in range (data_location, int(reading_end) + 1):
            body.append(get_line(filename, i, replace_units))

    body = np.array(body)
    return pd.DataFrame(body, columns=headers)

In [7]:
# test of previous implementation that it still works

get_means_table(os.path.join(src, 'Cum_Q.out'), 11, 14, 12).sample(3)

,Time_T,CumQAP_L3,CumQRP_L3,CumQA_L3,CumQR_L3,CumQ3_L3,CumQ1_L3,CumQS_L3,CumQ5_L3,CumQ6_L3,CumQ7_L3,CumQ8_L3,CumQ9_L3,CRunOff_L3,cEvapor_L3,cInfiltr_L3
160,5.3327,0.000E+00,0.000E+00,0.000E+00,0.000E+00,-0.213E+03,0.000E+00,0.000E+00,0.000E+00,0.121E-01,0.000E+00,0.000E+00,0.000E+00,0.000E+00,0.000E+00,0.000E+00
116,2.4271,0.000E+00,0.000E+00,0.000E+00,0.000E+00,-0.971E+02,0.000E+00,0.000E+00,0.000E+00,0.822E-02,0.000E+00,0.000E+00,0.000E+00,0.000E+00,0.000E+00,0.000E+00
542,272.8801,0.000E+00,0.000E+00,0.000E+00,0.000E+00,-0.240E+04,0.000E+00,0.000E+00,0.000E+00,0.329E+02,0.000E+00,0.000E+00,0.000E+00,0.000E+00,0.000E+00,0.000E+00


In [8]:
# test of the new implementation that it works

get_means_table(os.path.join(src, 'Boundary.out'), 19, 22, units_location=None,
    reading_end=539,replace_units=False).sample(3)

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ
210,211,2849,36.63,4.62,22.48,-4,0.000E+00,0.000E+00,-18.23,0.119,20.000,0.000E+00
53,54,650,15.65,20.00,21.64,-4,0.000E+00,0.000E+00,-28.59,0.081,20.000,0.000E+00
370,371,3009,36.63,12.31,22.48,-4,0.000E+00,0.000E+00,-15.88,0.136,20.000,0.000E+00


## Concatenating several dataframes for each timestep

In [9]:
df_t05 = get_means_table(
    os.path.join(src, 'Boundary.out'),
    19,
    22,
    units_location=None,
    reading_end=539,
    replace_units=False).sample(3)
df_t15 = get_means_table(
    os.path.join(src, 'Boundary.out'),
    544,
    547,
    units_location=None,
    reading_end=1064,
    replace_units=False).sample(3)

display(df_t05, df_t15)

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ
455,456,3094,20.14,16.92,21.82,-4,0.000E+00,0.000E+00,-28.69,0.080,20.000,0.000E+00
52,53,648,14.15,20.00,21.58,-4,0.000E+00,0.000E+00,-28.87,0.080,20.000,0.000E+00
348,349,2987,3.66,12.31,21.16,-4,0.000E+00,0.000E+00,-27.04,0.084,20.000,0.000E+00


,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ
144,145,2783,33.63,1.54,22.36,-4,0.000E+00,0.000E+00,-1.07,0.430,20.000,0.000E+00
215,216,2854,44.12,4.62,22.78,-4,0.000E+00,0.000E+00,4.34,0.430,20.000,0.000E+00
305,306,2944,35.13,9.23,22.42,-4,0.000E+00,0.000E+00,-0.32,0.430,20.000,0.000E+00


In [10]:
df_t05.assign(time_step= 5,inplace=True)#, df_t15['time_step']=15
display(df_t05, df_t15)

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ
455,456,3094,20.14,16.92,21.82,-4,0.000E+00,0.000E+00,-28.69,0.080,20.000,0.000E+00
52,53,648,14.15,20.00,21.58,-4,0.000E+00,0.000E+00,-28.87,0.080,20.000,0.000E+00
348,349,2987,3.66,12.31,21.16,-4,0.000E+00,0.000E+00,-27.04,0.084,20.000,0.000E+00


,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ
144,145,2783,33.63,1.54,22.36,-4,0.000E+00,0.000E+00,-1.07,0.430,20.000,0.000E+00
215,216,2854,44.12,4.62,22.78,-4,0.000E+00,0.000E+00,4.34,0.430,20.000,0.000E+00
305,306,2944,35.13,9.23,22.42,-4,0.000E+00,0.000E+00,-0.32,0.430,20.000,0.000E+00


In [11]:
df_t05 = df_t05.assign(time_step= 5)
df_t15 = df_t15.assign(time_step= 15)
display(df_t05, df_t15)

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step
455,456,3094,20.14,16.92,21.82,-4,0.000E+00,0.000E+00,-28.69,0.080,20.000,0.000E+00,5
52,53,648,14.15,20.00,21.58,-4,0.000E+00,0.000E+00,-28.87,0.080,20.000,0.000E+00,5
348,349,2987,3.66,12.31,21.16,-4,0.000E+00,0.000E+00,-27.04,0.084,20.000,0.000E+00,5


,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step
144,145,2783,33.63,1.54,22.36,-4,0.000E+00,0.000E+00,-1.07,0.430,20.000,0.000E+00,15
215,216,2854,44.12,4.62,22.78,-4,0.000E+00,0.000E+00,4.34,0.430,20.000,0.000E+00,15
305,306,2944,35.13,9.23,22.42,-4,0.000E+00,0.000E+00,-0.32,0.430,20.000,0.000E+00,15


In [12]:
pd.concat([df_t05, df_t15])

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step
455,456,3094,20.14,16.92,21.82,-4,0.000E+00,0.000E+00,-28.69,0.080,20.000,0.000E+00,5
52,53,648,14.15,20.00,21.58,-4,0.000E+00,0.000E+00,-28.87,0.080,20.000,0.000E+00,5
348,349,2987,3.66,12.31,21.16,-4,0.000E+00,0.000E+00,-27.04,0.084,20.000,0.000E+00,5
144,145,2783,33.63,1.54,22.36,-4,0.000E+00,0.000E+00,-1.07,0.430,20.000,0.000E+00,15
215,216,2854,44.12,4.62,22.78,-4,0.000E+00,0.000E+00,4.34,0.430,20.000,0.000E+00,15
305,306,2944,35.13,9.23,22.42,-4,0.000E+00,0.000E+00,-0.32,0.430,20.000,0.000E+00,15


### Read the dataframe of the simulation

In [13]:
df = read_hydrus_data(folder=src, save_to_csv=False, read_velocities=True)
display(df.sample(3), get_full_simulation_info(df))

df_rotated = rotate_back(df, 2.2899, rotation_axis='y')
get_full_simulation_info(df_rotated)

 For the entire simulation
Time steps : [0, 5, 15, 30, 60, 120, 180, 240, 500, 720, 1440] 
Dimensions : {'x': (-0.839105, 49.9601), 'y': (0.0, 20.0), 'z': (0.0, 22.9811)} 
Matrix dims: x_vals(41,), z_vals(46,), X(46, 41), Z(46, 41), M(46, 41)


 Variables statistics:


,n,x,y,z,Th_T0,Th_T5,Th_T15,Th_T30,Th_T60,Th_T120,...,V3_T240,V1_T500,V2_T500,V3_T500,V1_T720,V2_T720,V3_T720,V1_T1440,V2_T1440,V3_T1440
6987,6987,28.5295,7.75000,16.28960,0.20,0.200009,0.203181,0.333524,0.390000,0.363297,...,-0.000408,-0.000020,0.000002,-0.000184,-0.000020,0.000001,-0.000117,-0.000017,4.690200e-07,-0.000045
7778,7778,48.0005,6.97186,13.90860,0.20,0.200000,0.200000,0.200022,0.221825,0.295974,...,-0.000859,0.000013,0.000005,-0.000426,0.000011,0.000004,-0.000239,0.000004,1.017270e-06,-0.000056
1728,1728,11.9105,18.46150,2.47789,0.15,0.212785,0.219668,0.222311,0.382393,0.378323,...,0.000796,-0.000075,-0.000003,0.000317,-0.000067,-0.000003,0.000186,-0.000048,-1.130680e-06,0.000046


,Min,Mean,Median,Max
Th,0.049132,0.257289,3.117305e-01,0.430000
H,-676.912000,-66.645863,-1.962600e+01,12.429200
V1,-1.660930,-0.021988,-1.864930e-05,0.802749
V2,-0.263769,-0.000331,1.837060e-08,0.265775
V3,-1.086420,-0.002844,-2.684000e-05,0.777201


 For the entire simulation
Time steps : [0, 5, 15, 30, 60, 120, 180, 240, 500, 720, 1440] 
Dimensions : {'x': (0.0, 50.0), 'y': (0.0, 20.0), 'z': (0.0, 21.0)} 
Matrix dims: x_vals(41,), z_vals(43,), X(43, 41), Z(43, 41), M(43, 41)


 Variables statistics:


,Min,Mean,Median,Max
Th,0.049132,0.257289,3.117305e-01,0.430000
H,-676.912000,-66.645863,-1.962600e+01,12.429200
V1,-1.660930,-0.021988,-1.864930e-05,0.802749
V2,-0.263769,-0.000331,1.837060e-08,0.265775
V3,-1.086420,-0.002844,-2.684000e-05,0.777201


In [14]:
get_available_timesteps(df)

[0, 5, 15, 30, 60, 120, 180, 240, 500, 720, 1440]

**Pattern**
```
For T0005: 0017, 0019, 0022, 0539

For T0015: 0542, 0544, 0547, 1064

For T0030:   +3,   +2,   +3, +517

For T0030: 1067, 1069, 1072, 1589
```

In [15]:
titles_loc = 19
data_begins = 22
num_lines = 517
frames = []
for t in get_available_timesteps(df)[1:]:
    data_ends = data_begins + num_lines
    # print(titles_loc, data_begins, num_lines, data_ends)
    dft = get_means_table(
        os.path.join(src, 'Boundary.out'),
        titles_loc,
        data_begins,
        units_location=None,
        reading_end=data_ends,
        replace_units=False).sample(3)
    dft = dft.assign(time_step=t)
    frames.append(dft)
    titles_loc = data_ends + 5
    data_begins = titles_loc + 3
pd.concat(frames,ignore_index=True)

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step
0,381,3019,3.66,13.85,21.16,-4,0.000E+00,0.000E+00,-27.67,0.083,20.000,0.000E+00,5
1,287,2925,6.65,9.23,21.28,-4,0.000E+00,0.000E+00,-26.64,0.085,20.000,0.000E+00,5
2,12,26,47.12,20.00,22.90,-4,0.000E+00,0.000E+00,-0.49,0.430,20.000,0.000E+00,5
3,331,2969,24.64,10.77,22.00,-4,0.000E+00,0.000E+00,-6.72,0.305,20.000,0.000E+00,15
4,294,2932,17.15,9.23,21.70,-4,0.000E+00,0.000E+00,-31.85,0.075,20.000,0.000E+00,15
5,173,2811,27.64,3.08,22.12,-4,0.000E+00,0.000E+00,-4.18,0.385,20.000,0.000E+00,15
6,9,23,-0.84,20.00,20.98,-4,0.000E+00,0.000E+00,-33.33,0.073,20.000,0.000E+00,30
7,489,3127,21.64,18.46,21.88,-4,0.000E+00,0.000E+00,-5.99,0.329,20.000,0.000E+00,30
8,219,2857,0.66,6.15,21.04,-4,0.000E+00,0.000E+00,-37.63,0.068,20.000,0.000E+00,30
9,435,3073,36.63,15.38,22.48,-4,0.000E+00,0.000E+00,3.26,0.430,20.000,0.000E+00,60


## Adding the `surface area` to the above table

In [16]:
data_in.sample(3)

,n,surface_area
250,2889,2.305850
7,21,0.506409
126,2765,2.305850


In [32]:
titles_loc = 19
data_begins = 22
num_lines = num_cells
frames = []
for t in get_available_timesteps(df)[1:]:
    data_ends = data_begins + num_lines -1
    print(titles_loc, data_begins, num_lines, data_ends)
    dft = get_means_table(
        os.path.join(src, 'Boundary.out'),
        titles_loc,
        data_begins,
        units_location=None,
        reading_end=data_ends,
        replace_units=False).sample(3)
    # add a column for time
    dft = dft.assign(time_step=t)
    # convert all columns to numeric
    cols = list(dft)
    dft[cols] = dft[cols].apply(pd.to_numeric, errors='coerce')
    dft = pd.merge(dft, data_in, on='n')
    frames.append(dft)
    titles_loc = data_ends + 5
    data_begins = titles_loc + 3
# display(frames)
pd.concat(frames,ignore_index=True)

19 22 518 539
544 547 518 1064
1069 1072 518 1589
1594 1597 518 2114
2119 2122 518 2639
2644 2647 518 3164
3169 3172 518 3689
3694 3697 518 4214
4219 4222 518 4739
4744 4747 518 5264


,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step,surface_area
0,248,2886,44.12,6.15,22.78,-4,0.000000,0.000000,-2.52,0.423,20.0,0.0000,5,2.305850
1,253,2891,3.66,7.69,21.16,-4,0.000000,0.000000,-27.48,0.083,20.0,0.0000,5,2.305850
2,403,3041,36.63,13.85,22.48,-4,0.000000,0.000000,-15.65,0.138,20.0,0.0000,5,2.305850
3,506,3144,47.12,18.46,22.90,-4,0.000000,0.000000,6.28,0.430,20.0,0.0000,15,2.691380
4,64,666,30.64,20.00,22.24,-4,0.000000,0.000000,-2.55,0.423,20.0,0.0000,15,1.152930
5,428,3066,26.14,15.38,22.06,-4,0.000000,0.000000,-5.32,0.350,20.0,0.0000,15,2.305850
6,122,2460,-0.84,18.46,20.98,-4,0.000000,0.000000,-35.26,0.070,20.0,0.0000,30,1.922150
7,131,2769,12.65,1.54,21.52,-4,0.000000,0.000000,-15.45,0.140,20.0,0.0000,30,2.305850
8,329,2967,21.64,10.77,21.88,-4,0.000000,0.000000,-5.96,0.330,20.0,0.0000,30,2.305850
9,200,2838,20.14,4.62,21.82,-4,0.000000,0.000000,-3.51,0.403,20.0,0.0000,60,2.305850


## Creating a combined function for both files

### Creating a saving function

In [73]:
def save_this(dataframe, data_folder, output_name, authorized=True):
    """
    if the `authorized` flag is true, the `dataframe` will be saved as CSV 
        file, in the location `data_folder`//Nesr with the name `output_name`
    """

    
    if authorized:
        # check for the output folder
        save_folder = os.path.join(data_folder, 'Nesr')
        if not os.path.exists(save_folder):
            os.makedirs(save_folder) 
        # Saving
        filename = output_name
        if filename[-4:].lower() != '.csv':
            filename = f'{filename}.CSV'
        dataframe.to_csv(
            os.path.join(save_folder, filename), index=False)

In [67]:
save_this(df, src, 'output_name', authorized=True)

In [74]:
'boundary_data.CSV'[-4:].lower() != '.csv'

False

### Create the combined function

In [69]:
def read_boundary_data(folder='Current',
                       titles_loc=19,
                       data_begins=22,
                       nums_per_line=10,
                       save_to_csv=True):
    '''
    A function to read both BOUNDARY.IN and  Boundary.outfiles from HYDRUS 
        outputs, then to:
            1- return one dataframe contains both data in a decent format.
            2- save this output to a CSV file (optional, True by default)
    Input:
        The name of the main folder (leave balank for the current folder)
        titles_loc=19, the line number of the first line contains captions
        data_begins=22, the line number of the first line contains data
        nums_per_line=10, teh number of columns per line in BOUNDARY.IN file
        The option to save_to csv, default =True (Boolean)
    '''

    # Specify the source folder
    if folder == 'Current':
        read_dir = os.getcwd()
    else:
        read_dir = folder

    # Finding number of nodes in the file
    mesh_file = os.path.join(read_dir, 'BOUNDARY.IN')
    num_cells = int(linecache.getline(mesh_file, 4).split()[0])
    # Define dataframe titles
    titles = ['n', 'surface_area']

    def get_num_lines(num_cells, nums_per_line=10):
        num_lines = int(num_cells / nums_per_line)
        if num_cells % nums_per_line > 0: num_lines += 1
        return num_lines

    def read_snakey_list(file_name, start_line, end_line, data_type=float):
        '''
        Reading a list of numbers thar are stored in a text file, where
        there are `numbers_count` numbers, stored as `nums_per_line` numbers 
        per line, the starting position of first occurence of numbers is at
        `start_line` line number
        returns a numpy array of the data with the `data_type` given
        '''
        points = []
        for i in range(start_line, end_line):
            points.extend(linecache.getline(file_name, i).split())
        return np.array(points, data_type)

    num_lines = get_num_lines(num_cells, nums_per_line)

    first_line = 10
    end_line = first_line + num_lines
    points = read_snakey_list(mesh_file, first_line, end_line, data_type=int)

    first_line = end_line + 1
    end_line = first_line + num_lines
    areas = read_snakey_list(mesh_file, first_line, end_line)
    data = np.array([points, areas]).T
    data_in = pd.DataFrame(
        data, columns=titles).astype({
            "n": int,
            "surface_area": float
        })

    # Adding the surface area to the above table
    # print(num_cells, nums_per_line, num_lines)
    frames = []
    for t in get_available_timesteps(df)[1:]:
        data_ends = data_begins + num_cells - 1
        # print(titles_loc, data_begins, num_lines, data_ends)
        dft = get_means_table(
            os.path.join(src, 'Boundary.out'),
            titles_loc,
            data_begins,
            units_location=None,
            reading_end=data_ends,
            replace_units=False)  #.sample(3)
        # add a column for time
        dft = dft.assign(time_step=t)
        # convert all columns to numeric
        cols = list(dft)
        # print(cols)
        # display(dft)
        dft[cols] = dft[cols].apply(pd.to_numeric, errors='coerce')
        dft = pd.merge(dft, data_in, on='n')
        frames.append(dft)
        titles_loc = data_ends + 5
        data_begins = titles_loc + 3
    # display(frames)
    frames = pd.concat(frames, ignore_index=True)
    # Saving if allowed

    #     if save_to_csv:
    #         # check for the output folder
    #         if not os.path.exists(os.path.join(src, 'Nesr')):
    #             os.makedirs(os.path.join(src, 'Nesr'))
    #         # Saving
    #         frames.to_csv(
    #             os.path.join(src, 'Nesr', 'boundary_data.CSV'), index=False)

    # save_this(dataframe, data_folder, output_name, authorized=True)
    save_this(frames, src, 'boundary_data.CSV', authorized=save_to_csv)
    return frames

In [70]:
read_boundary_data(src, 19, 22)
# read, displayed, and saved to the Nesr sub-folder

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step,surface_area
0,1,1,0.00,0.00,0.00,-6,-9.890000e-09,4.830000e-07,-57.69,0.056,20.0,-1.810000e-06,5,0.020491
1,2,8,-0.08,0.00,2.00,-6,-1.500000e-08,3.660000e-07,-164.11,0.198,20.0,-4.710000e-08,5,0.040982
2,3,9,-0.80,0.00,19.98,-6,-7.430000e-05,9.740000e-05,-30.62,0.077,20.0,-8.150000e-04,5,0.762819
3,4,11,-0.84,0.00,20.98,-4,0.000000e+00,0.000000e+00,-27.31,0.083,20.0,0.000000e+00,5,1.275030
4,5,12,49.12,0.00,22.98,-3,1.030000e+00,-2.000000e+00,0.48,0.430,20.0,5.130000e+00,5,0.512820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5175,514,3152,49.12,7.69,22.98,-3,0.000000e+00,0.000000e+00,-25.88,0.087,20.0,1.850000e+02,1440,1.538460
5176,515,3153,49.12,6.15,22.98,-3,0.000000e+00,0.000000e+00,-25.87,0.087,20.0,1.850000e+02,1440,1.538460
5177,516,3154,49.12,4.62,22.98,-3,0.000000e+00,0.000000e+00,-25.87,0.087,20.0,1.850000e+02,1440,1.538460
5178,517,3155,49.12,3.08,22.98,-3,0.000000e+00,0.000000e+00,-25.87,0.087,20.0,1.850000e+02,1440,1.538460
